In [ ]:
%pip install pandas matplotlib seaborn scikit-learn

In [ ]:
# Importa bibliotecas principais
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import sys

# Configurações para melhor visualização
pd.set_option('display.max_columns', None)
sns.set_theme(style="whitegrid")

# Adiciona o caminho da pasta 'src' ao sistema
# (../ sobe um nível, da pasta 'notebooks' para a raiz 'tech_challenge_voos')
sys.path.append('../') 

# Importa nossa função customizada de carregamento
from src.data_loader import load_datasets
from src.preprocessing import clean_flights 
from src.feature_engineering import create_target_variable
from src.modeling import create_preprocessing_pipeline

print("Bibliotecas e módulos importados com sucesso!")

In [ ]:
# Carrega os dados usando a função do script .py
# (O caminho '../data/' sobe um nível e entra na pasta 'data')
df_flights, df_airlines, df_airports = load_datasets('../data/')

In [ ]:
# Agora sim, a primeira análise!
if df_flights is not None:
    print("--- Informações de df_flights ---")
    df_flights.info()
else:
    print("Erro: DataFrame df_flights não foi carregado.")

In [ ]:
# Célula 4: Verificando valores ausentes
if df_flights is not None:
    print("Contagem de valores ausentes (nulos) por coluna:")
    print(df_flights.isnull().sum())

In [ ]:
# Célula 5: Aplicando a limpeza
if df_flights is not None:
    print(f"Tamanho original: {df_flights.shape}")
    
    df_flights_clean = clean_flights(df_flights)
    
    print(f"Tamanho limpo:    {df_flights_clean.shape}")
else:
    print("df_flights não carregado.")

In [ ]:
# Célula 6: Verificação final de nulos
if 'df_flights_clean' in locals():
    print("\nVerificando nulos no dataset limpo:")
    # .sum().sum() soma todos os nulos de todas as colunas
    print(df_flights_clean.isnull().sum().sum())
else:
    print("df_flights_clean não foi criado.")

In [ ]:
# Célula 7: Estatísticas Descritivas (Numéricas)
if 'df_flights_clean' in locals():
    print("Estatísticas Descritivas (Dados Numéricos):")
    display(df_flights_clean.describe())
else:
    print("df_flights_clean não foi criado.")

In [ ]:
# Célula 8: Visualizando a distribuição dos atrasos na chegada
if 'df_flights_clean' in locals():
    print("Plotando a distribuição dos Atrasos na Chegada (ARRIVAL_DELAY)")
    
    plt.figure(figsize=(12, 6))
    
    # Usamos o `df_flights_clean`
    sns.histplot(df_flights_clean['ARRIVAL_DELAY'], bins=100, kde=True)
    
    # Linha vertical vermelha para a média (4.4 min)
    plt.axvline(
        df_flights_clean['ARRIVAL_DELAY'].mean(), 
        color='red', 
        linestyle='--', 
        linewidth=2, 
        label=f'Média ({df_flights_clean["ARRIVAL_DELAY"].mean():.2f})'
    )
    
    # Linha vertical verde para a mediana (-5.0 min)
    plt.axvline(
        df_flights_clean['ARRIVAL_DELAY'].median(), 
        color='green', 
        linestyle='-', 
        linewidth=2, 
        label=f'Mediana ({df_flights_clean["ARRIVAL_DELAY"].median():.2f})'
    )
    
    # Dando "zoom" em uma área de interesse
    # (Sem isso, o gráfico ficaria ilegível por causa dos outliers)
    plt.xlim(-60, 180) 
    
    plt.title('Distribuição dos Atrasos na Chegada (Zoom de -60 a 180 min)', fontsize=16)
    plt.xlabel('Atraso na Chegada (Minutos)')
    plt.ylabel('Contagem de Voos')
    plt.legend()
    plt.show()

else:
    print("df_flights_clean não foi criado.")

In [ ]:
# Célula 9: Criando a Variável Alvo (Target)
if 'df_flights_clean' in locals():
    
    # Usamos a função do nosso script .py
    df_model_data = create_target_variable(df_flights_clean, delay_threshold=15)
    
    print("\nVerificando a distribuição da nova coluna 'IS_DELAYED':")
    # Mostra a contagem (0 = Não Atrasado, 1 = Atrasado) e a porcentagem
    display(df_model_data['IS_DELAYED'].value_counts(normalize=True))
    
else:
    print("df_flights_clean não foi criado.")

In [ ]:
# Célula 10 (CORRIGIDA): Importações para Modelagem

import sys
import os

# Esta linha diz ao Python para "subir um nível" (../) 
# e procurar por módulos lá. É o que permite encontrar a pasta 'src'.
sys.path.append(os.path.abspath(os.path.join(os.getcwd(), '..')))

# Agora estas importações vão funcionar
from sklearn.model_selection import train_test_split
from src.modeling import create_preprocessing_pipeline
from src.feature_engineering import engineer_time_features

print("Importações de modelagem carregadas com sucesso!")

In [ ]:
import pandas as pd
import numpy as np

# --- CÉLULA 1: Criar as colunas de Engenharia de Feature ---

try:
    print("Criando 'HOUR_OF_DAY' e 'PART_OF_DAY'...")
    
    # 1. Criar a coluna 'HOUR_OF_DAY'
    # // 100 faz a divisão inteira (ex: 1150 -> 11). 
    # % 24 garante que a hora '24' (meia-noite) vire '0'.
    df_flights_clean['HOUR_OF_DAY'] = (df_flights_clean['SCHEDULED_DEPARTURE'] // 100) % 24

    # 2. Definir os "cortes" (bins) e os "rótulos" (labels)
    # Bins:   -1 a 5 (Madrugada), 6 a 11 (Manhã), 12 a 17 (Tarde), 18 a 23 (Noite)
    bins = [-1, 5, 11, 17, 23]
    labels = ['Madrugada', 'Manhã', 'Tarde', 'Noite']

    # 3. Criar a coluna 'PART_OF_DAY'
    df_flights_clean['PART_OF_DAY'] = pd.cut(
        df_flights_clean['HOUR_OF_DAY'], 
        bins=bins, 
        labels=labels, 
        right=True
    )
    
    print("-> Colunas 'HOUR_OF_DAY' e 'PART_OF_DAY' criadas com sucesso!")
    print("\n--- PREPARAÇÃO DO EIXO X CONCLUÍDA ---")

except KeyError:
    print("Erro: A coluna 'SCHEDULED_DEPARTURE' não foi encontrada.")
except Exception as e:
    print(f"Ocorreu um erro ao criar PART_OF_DAY: {e}")

In [ ]:
import pandas as pd
import numpy as np

print("--- Célula 10.5: Preparação Final para Modelagem ---")

try:
    # --- 1. Criar Features de Engenharia ---
    # (Este é o código que estávamos depurando)
    print("Criando 'HOUR_OF_DAY' e 'PART_OF_DAY'...")
    
    # Garante que a coluna de hora está no formato certo
    df_flights_clean['SCHEDULED_DEPARTURE'] = df_flights_clean['SCHEDULED_DEPARTURE'].astype(int)
    
    # // 100 faz a divisão inteira (ex: 1150 -> 11). 
    df_flights_clean['HOUR_OF_DAY'] = (df_flights_clean['SCHEDULED_DEPARTURE'] // 100) % 24
    
    # Define os "cortes" (bins) e os "rótulos" (labels)
    bins = [-1, 5, 11, 17, 23]
    labels = ['Madrugada', 'Manhã', 'Tarde', 'Noite']

    # Cria a coluna 'PART_OF_DAY'
    df_flights_clean['PART_OF_DAY'] = pd.cut(
        df_flights_clean['HOUR_OF_DAY'], 
        bins=bins, 
        labels=labels, 
        right=True
    )
    print("-> Features de engenharia criadas.")

    # --- 2. Criar a Variável Alvo (Target) ---
    print("Criando a variável alvo 'IS_DELAYED'...")
    LIMITE_ATRASO = 15 
    
    # (Vou usar 'IS_DELAYED' pois vi esse nome no seu código da Célula 11)
    df_flights_clean['IS_DELAYED'] = (df_flights_clean['ARRIVAL_DELAY'] > LIMITE_ATRASO).astype(int)
    print("-> Variável alvo 'IS_DELAYED' criada.")

    # --- 3. Definir as colunas FINAIS ---
    # Esta é a lista das 9 features + 1 target que o Modelo 11 espera
    FINAL_COLUMNS = [
        # Numéricas
        'MONTH', 
        'DAY_OF_WEEK', 
        'SCHEDULED_TIME', 
        'DISTANCE',
        'HOUR_OF_DAY',
        # Categóricas
        'AIRLINE', 
        'ORIGIN_AIRPORT', 
        'DESTINATION_AIRPORT',
        'PART_OF_DAY',
        # Alvo
        'IS_DELAYED' 
    ]
    
    # --- 4. Criar o DataFrame 'df_model_data_v2' ---
    # Seleciona apenas as colunas que vamos usar e remove Nulos
    # (O .dropna() aqui é uma segurança final, caso alguma feature nova tenha nulos)
    df_model_data_v2 = df_flights_clean[FINAL_COLUMNS].dropna()

    print("\n--- SUCESSO! ---")
    print("DataFrame 'df_model_data_v2' criado com sucesso.")
    print(f"Formato (shape) do dataset de modelagem: {df_model_data_v2.shape}")
    display(df_model_data_v2.head())

except KeyError as e:
    print(f"\n--- ERRO ---")
    print(f"Erro de Chave (KeyError): A coluna {e} não foi encontrada.")
    print("Certifique-se de que 'df_flights_clean' existe e tem as colunas originais (ex: 'SCHEDULED_DEPARTURE', 'ARRIVAL_DELAY').")
except Exception as e:
    print(f"\nOcorreu um erro inesperado: {e}")

In [ ]:
# --- CÉLULA 1: Criar a coluna-alvo (target) ---

# O desafio pede para prever se um voo vai atrasar.
# Vamos usar o padrão da indústria: um voo "atrasado" é aquele que chega
# 15 minutos ou mais após o horário programado.
LIMITE_ATRASO = 15 

try:
    # Cria a nova coluna. (df['col'] > X) vira True/False.
    # .astype(int) transforma True/False em 1/0, o que é perfeito para cálculos.
    df_flights_clean['ATRASOU_MAIS_15MIN'] = (df_flights_clean['ARRIVAL_DELAY'] > LIMITE_ATRASO).astype(int)
    
    print("Coluna 'ATRASOU_MAIS_15MIN' criada com sucesso!")
    
    # Vamos verificar a proporção (é um dataset desbalanceado?)
    # .value_counts(normalize=True) mostra a porcentagem
    print("\nProporção de voos Atrasados (1) vs. Não Atrasados (0):")
    print(df_flights_clean['ATRASOU_MAIS_15MIN'].value_counts(normalize=True))

except KeyError:
    print("Erro: A coluna 'ARRIVAL_DELAY' não foi encontrada. Verifique os passos anteriores.")
except Exception as e:
    print(f"Ocorreu um erro: {e}")

In [ ]:
# --- CÉLULA 2: Gerar o gráfico (Agora vai funcionar) ---
import seaborn as sns
import matplotlib.pyplot as plt

# Define a ordem que você quer no eixo X
ordem_partes_dia = ['Madrugada', 'Manhã', 'Tarde', 'Noite']

plt.figure(figsize=(10, 6))

# O sns.barplot, por padrão, calcula a MÉDIA de 'y' para cada 'x'
# Como 'y' é 0 ou 1, a média (ex: 0.22) é o mesmo que o percentual (22%)!
sns.barplot(
    x="PART_OF_DAY",
    y="ATRASOU_MAIS_15MIN", 
    data=df_flights_clean,
    order=ordem_partes_dia,
    errorbar=None
)

plt.title('Percentual de Voos com Atraso (>15min) por Período do Dia')
plt.ylabel('Taxa de Atraso (Média de 0s e 1s)')
plt.xlabel('Período do Dia')
plt.show()

In [ ]:
# Célula 11 (MODIFICADA - VERSÃO CORRETA COM 9 FEATURES)

if 'df_model_data_v2' in locals():
    
    # 1. Definir NOVAS colunas de features (X)
    numeric_features = [
        'MONTH', 
        'DAY_OF_WEEK', 
        'SCHEDULED_TIME', 
        'DISTANCE',
        'HOUR_OF_DAY'  # <-- Feature 5
    ]
    
    categorical_features = [
        'AIRLINE',             # Feature 6
        'ORIGIN_AIRPORT',      # Feature 7
        'DESTINATION_AIRPORT', # Feature 8
        'PART_OF_DAY'          # <-- Feature 9
    ]
    
    TARGET = 'IS_DELAYED'
    
    # Garantir que as categóricas são strings (para o encoder)
    df_model_ready = df_model_data_v2.copy()
    for col in categorical_features:
        df_model_ready[col] = df_model_ready[col].astype(str)
            
    # 2. Definir X e y a partir dos dados v2
    FEATURES = numeric_features + categorical_features
    X = df_model_ready[FEATURES]
    y = df_model_ready[TARGET]
    
    # 3. Dividir os dados
    X_train, X_test, y_train, y_test = train_test_split(
        X, y, 
        test_size=0.2, 
        random_state=42,
        stratify=y
    )
    
    print("Dados divididos em Treino e Teste (com Novas Features):")
    print(f"X_train shape: {X_train.shape}") # <-- DEVE MOSTRAR (..., 9)
    print(f"X_test shape:  {X_test.shape}") # <-- DEVE MOSTRAR (..., 9)
    
else:
    print("Erro: df_model_data_v2 não foi criado. Rode a célula 'Nova Célula' (C-10.5) primeiro.")

In [ ]:
# Célula 12: Criando o pipeline de pré-processamento
if 'numeric_features' in locals() and 'categorical_features' in locals():
    
    # Usamos a função do nosso script .py
    preprocessor = create_preprocessing_pipeline(numeric_features, categorical_features)
    
    print("\nDefinição do Pré-processador:")
    display(preprocessor)
    
else:
    print("Erro: Listas 'numeric_features' ou 'categorical_features' não definidas.")
    print("Por favor, rode a Célula 11 primeiro.")

In [ ]:
# Célula 13: Criar e Treinar o Pipeline (Baseline - LogisticRegression)

# 1. Importações necessárias
from sklearn.pipeline import Pipeline
from sklearn.linear_model import LogisticRegression
import time # Para marcar o tempo de treino

# 2. Criar o Pipeline completo
# Ele combina o pré-processador (passo 'preprocessor')
# com o modelo de classificação (passo 'classifier')
model_lr_pipe = Pipeline(steps=[
    ('preprocessor', preprocessor), # Nosso ColumnTransformer da Célula 12
    ('classifier', LogisticRegression(random_state=42, max_iter=1000)) 
    # max_iter=1000 ajuda o modelo a convergir com mais dados
])

# 3. Treinar o modelo!
print("Iniciando o treinamento do modelo LogisticRegression...")
start_time = time.time()

# O .fit() aqui dispara todo o processo (Scaler, OneHotEncoder E o treino)
model_lr_pipe.fit(X_train, y_train) 

end_time = time.time()
print(f"Treinamento do LogisticRegression concluído em {(end_time - start_time):.2f} segundos.")

print("\nModelo baseline (LogisticRegression) treinado com sucesso!")
display(model_lr_pipe)

In [ ]:
# Célula 14: Avaliando o Modelo Baseline (LogisticRegression)

# 1. Importações necessárias
from sklearn.metrics import classification_report, ConfusionMatrixDisplay
import matplotlib.pyplot as plt

print("Avaliando o modelo LogisticRegression nos dados de TESTE...")

# 2. Fazer previsões nos dados de teste (X_test)
# O .predict() aqui dispara todo o pipeline (Scaler, OneHotEncoder E a previsão)
y_pred_lr = model_lr_pipe.predict(X_test)

# 3. Gerar o Relatório de Classificação
print("\nRelatório de Classificação (LogisticRegression):")
# target_names dá nomes mais claros para '0' e '1'
report_lr = classification_report(
    y_test, 
    y_pred_lr, 
    target_names=['0 (Não Atrasado)', '1 (Atrasado)']
)
print(report_lr)

# 4. Gerar a Matriz de Confusão
print("Plotando Matriz de Confusão (LogisticRegression):")

fig, ax = plt.subplots(figsize=(8, 6))
# Plota a matriz usando os valores reais (y_test) vs. os previstos (y_pred_lr)
ConfusionMatrixDisplay.from_predictions(
    y_test, 
    y_pred_lr, 
    ax=ax, 
    cmap='Blues',
    display_labels=['Não Atrasado', 'Atrasado']
)
plt.title('Matriz de Confusão - LogisticRegression')
plt.show()

In [ ]:
# Célula 15 (CORRIGIDA E COMPLETA): Treinar e Avaliar Modelo 2

# --- Importações necessárias (resolvendo NameErrors) ---
import time
from sklearn.pipeline import Pipeline
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report, ConfusionMatrixDisplay
import matplotlib.pyplot as plt
# --------------------------------------------------------

print("Iniciando o treinamento do Modelo 2 (LogisticRegression com class_weight='balanced')")
start_time = time.time()

# 1. Criar um NOVO pipeline
# (Certifique-se de que 'preprocessor', X_train, y_train existem das células 11 e 12)
model_lr_balanced_pipe = Pipeline(steps=[
    ('preprocessor', preprocessor),
    ('classifier', LogisticRegression(
        random_state=42, 
        max_iter=1000,
        class_weight='balanced'
    )) 
])

# 2. Treinar o novo modelo
model_lr_balanced_pipe.fit(X_train, y_train) 

end_time = time.time()
print(f"Treinamento concluído em {(end_time - start_time):.2f} segundos.")

# 3. Avaliar o novo modelo
print("\nAvaliando o modelo (LogisticRegression com class_weight='balanced') nos dados de TESTE...")

y_pred_lr_balanced = model_lr_balanced_pipe.predict(X_test)

print("\nRelatório de Classificação (LR com class_weight):")
report_lr_balanced = classification_report(
    y_test, 
    y_pred_lr_balanced, 
    target_names=['0 (Não Atrasado)', '1 (Atrasado)']
)
print(report_lr_balanced)

# 4. Nova Matriz de Confusão
print("Plotando Matriz de Confusão (LR com class_weight):")
fig, ax = plt.subplots(figsize=(8, 6))
ConfusionMatrixDisplay.from_predictions(
    y_test, 
    y_pred_lr_balanced, 
    ax=ax, 
    cmap='Blues',
    display_labels=['Não Atrasado', 'Atrasado']
)
plt.title('Matriz de Confusão - LogisticRegression (class_weight=balanced)')
plt.show()

In [ ]:
# Célula 15.5: Verificando as Features do Modelo Treinado

print("Inspecionando as features que o modelo (Célula 15) realmente usou:")

try:
    # 1. Pega os nomes das features DEPOIS do ColumnTransformer
    # (ex: 'num__HOUR_OF_DAY', 'cat__PART_OF_DAY_Manhã')
    feature_names = model_lr_balanced_pipe.named_steps['preprocessor'].get_feature_names_out()
    
    # 2. Pega os coeficientes (importância) do modelo LogReg
    coefficients = model_lr_balanced_pipe.named_steps['classifier'].coef_[0]
    
    # 3. Cria um DataFrame para visualização fácil
    coef_df = pd.DataFrame({
        'Feature': feature_names,
        'Coefficient': coefficients
    }).sort_values(by='Coefficient', ascending=False)
    
    print("Top 10 features que o modelo mais usou:")
    display(coef_df.head(10))
    
    print("\nÚltimas 10 features que o modelo usou:")
    display(coef_df.tail(10))
    
    # 4. Verificação final
    if 'num__SCHEDULED_DEPARTURE' in coef_df['Feature'].values:
        print("\n--- VEREDITO ---")
        print("FALHA NO PIPELINE: O modelo usou a feature ANTIGA ('SCHEDULED_DEPARTURE').")
        print("Causa provável: A Célula 11 ou 12 não foi re-executada antes da Célula 15.")
        
    elif 'num__HOUR_OF_DAY' in coef_df['Feature'].values:
        print("\n--- VEREDITO ---")
        print("PIPELINE CORRETO: O modelo usou as features NOVAS ('HOUR_OF_DAY', 'PART_OF_DAY').")
        print("Conclusão: As novas features não melhoraram o desempenho do LogisticRegression.")
    else:
        print("\n--- VEREDITO ---")
        print("Verifique a lista de features manualmente.")

except Exception as e:
    print(f"Ocorreu um erro ao inspecionar o modelo: {e}")
    print("Por favor, certifique-se de que as Células 11, 12 e 15 foram executadas.")

In [ ]:
# Célula 16: Preparando Dados para Clusterização (Não Supervisionado)

# 1. Importações necessárias
from sklearn.preprocessing import StandardScaler
from sklearn.cluster import KMeans

print("Preparando dados para clusterização de aeroportos...")

# 2. Criar um DataFrame de 'features' por aeroporto
# Vamos focar nos aeroportos de ORIGEM
airport_features = df_flights_clean.groupby('ORIGIN_AIRPORT').agg(
    
    # Contagem total de voos saindo
    total_flights=('FLIGHT_NUMBER', 'count'),
    
    # Média de atraso na SAÍDA
    avg_departure_delay=('DEPARTURE_DELAY', 'mean'),
    
    # Porcentagem de voos que saíram atrasados (> 15 min)
    pct_delayed_departures=('DEPARTURE_DELAY', lambda x: (x > 15).mean())
    
).reset_index()


# 3. Escalonar (Padronizar) os dados
# K-Means é muito sensível à escala. Não podemos comparar 'total_flights' 
# (que vai a 100.000) com 'pct_delayed' (que vai de 0 a 1).
scaler = StandardScaler()

# Salvamos os nomes dos aeroportos antes de escalonar
airport_names = airport_features['ORIGIN_AIRPORT']

# Removemos o nome para escalar apenas os números
features_to_scale = airport_features.drop(columns=['ORIGIN_AIRPORT'])

# Escalona
features_scaled = scaler.fit_transform(features_to_scale)

print("\nFeatures dos aeroportos prontas e escalonadas.")
print(f"Total de aeroportos únicos: {len(airport_features)}")
display(airport_features.head())

In [ ]:
# Célula 17: Método do Cotovelo (Elbow Method) para encontrar o K ideal

if 'features_scaled' in locals():
    print("Executando o Método do Cotovelo para K de 1 a 10...")
    
    inertia_list = []  # Lista para guardar a inércia de cada K
    k_range = range(1, 11) # Vamos testar K de 1 a 10
    
    for k in k_range:
        # Cria e treina o modelo K-Means para o 'k' atual
        kmeans = KMeans(
            n_clusters=k, 
            random_state=42, 
            n_init=10 # n_init=10 para suprimir warnings
        )
        kmeans.fit(features_scaled)
        
        # Adiciona a inércia (WCSS) à lista
        inertia_list.append(kmeans.inertia_)
    
    # Plotar o gráfico do cotovelo
    print("Plotando o gráfico do Método do Cotovelo...")
    plt.figure(figsize=(10, 6))
    plt.plot(k_range, inertia_list, marker='o', linestyle='--')
    plt.xlabel('Número de Clusters (K)')
    plt.ylabel('Inertia (WCSS)')
    plt.title('Método do Cotovelo (Elbow Method) para K-Means')
    plt.xticks(k_range)
    plt.grid(True)
    plt.show()

else:
    print("Erro: 'features_scaled' não foi criado. Rode a Célula 16 primeiro.")

In [ ]:
# Célula 18: Rodando K-Means (K=4) e Analisando os Clusters

if 'features_scaled' in locals() and 'airport_features' in locals():
    
    # 1. Definir o número de clusters com base no nosso "Cotovelo"
    K_IDEAL = 4
    
    # 2. Rodar o modelo K-Means final
    kmeans = KMeans(
        n_clusters=K_IDEAL, 
        random_state=42, 
        n_init=10
    )
    kmeans.fit(features_scaled)
    
    # 3. Adicionar os rótulos dos clusters de volta ao DataFrame original
    airport_features_clustered = airport_features.copy()
    airport_features_clustered['cluster'] = kmeans.labels_
    
    print(f"K-Means executado com K={K_IDEAL}.")
    print("DataFrame com clusters:")
    display(airport_features_clustered.head())
    
    # 4. Interpretar os Clusters
    # (Esta é a parte mais importante!)
    # Vamos calcular a média de cada feature para cada cluster
    print("\n--- Interpretação dos Clusters (Médias) ---")
    cluster_interpretation = airport_features_clustered.groupby('cluster').agg({
        'total_flights': 'mean',
        'avg_departure_delay': 'mean',
        'pct_delayed_departures': 'mean',
        'ORIGIN_AIRPORT': 'count'  # Para ver quantos aeroportos em cada cluster
    }).rename(columns={'ORIGIN_AIRPORT': 'airport_count'}).sort_values(by='total_flights')
    
    display(cluster_interpretation)
    
    # 5. Visualizar os Clusters (Requisito Obrigatório)
    print("\n--- Visualização dos Clusters ---")
    
    # Vamos plotar Atraso Médio vs. % de Voos Atrasados
    # O tamanho (size) do ponto será o Total de Voos
    plt.figure(figsize=(12, 8))
    sns.scatterplot(
        data=airport_features_clustered,
        x='avg_departure_delay',
        y='pct_delayed_departures',
        hue='cluster',         # Cor por cluster
        size='total_flights',  # Tamanho por volume de voos
        sizes=(50, 2000),      # Range dos tamanhos
        palette='viridis',     # Esquema de cores
        alpha=0.7
    )
    
    plt.title(f'Clusterização de Aeroportos por Perfil de Atraso (K={K_IDEAL})', fontsize=16)
    plt.xlabel('Atraso Médio na Partida (min)')
    plt.ylabel('Porcentagem de Voos Atrasados (>15 min)')
    plt.legend(title='Cluster', bbox_to_anchor=(1.05, 1), loc='upper left')
    plt.grid(True)
    plt.show()

else:
    print("Erro: 'features_scaled' ou 'airport_features' não criados. Rode as células anteriores.")

In [ ]:
# Célula 19: Treinar o Modelo 3 (RandomForestClassifier em uma Amostra)

# 1. Importações necessárias
from sklearn.ensemble import RandomForestClassifier

# 2. Criar uma amostra de 10% dos dados de treino (para velocidade)
# RandomForest é computacionalmente caro. Vamos usar 450k linhas.
print("Criando amostra de 10% dos dados de treino (aprox. 450k linhas)...")
X_train_sample, _, y_train_sample, _ = train_test_split(
    X_train, y_train, 
    train_size=0.1,  # 10% dos dados
    random_state=42, 
    stratify=y_train # Mantém a proporção de 81/19 na amostra
)

print(f"Tamanho da amostra de treino: {X_train_sample.shape}")

# 3. Criar o Pipeline completo
print("Iniciando o treinamento do Modelo 3 (RandomForestClassifier)...")
start_time = time.time()

model_rf_pipe = Pipeline(steps=[
    ('preprocessor', preprocessor),
    ('classifier', RandomForestClassifier(
        random_state=42, 
        class_weight='balanced', # Crucial para o desbalanceamento
        n_jobs=-1                # Usar todos os processadores
    )) 
])

# 4. Treinar o modelo (NA AMOSTRA!)
model_rf_pipe.fit(X_train_sample, y_train_sample) 

end_time = time.time()
print(f"Treinamento do RandomForest concluído em {(end_time - start_time):.2f} segundos.")

In [ ]:
# Célula 20: Avaliando o Modelo 3 (RandomForestClassifier)

print("\nAvaliando o modelo (RandomForestClassifier) nos dados de TESTE COMPLETOS...")

# 1. Importações (caso o kernel tenha sido reiniciado)
from sklearn.metrics import classification_report, ConfusionMatrixDisplay
import matplotlib.pyplot as plt

# 2. Fazer previsões (no X_test completo)
# (Certifique-se que model_rf_pipe existe da Célula 19)
y_pred_rf = model_rf_pipe.predict(X_test)

# 3. Gerar o Relatório de Classificação
print("\nRelatório de Classificação (RandomForestClassifier):")
report_rf = classification_report(
    y_test, 
    y_pred_rf, 
    target_names=['0 (Não Atrasado)', '1 (Atrasado)']
)
print(report_rf)

# 4. Gerar a Matriz de Confusão
print("Plotando Matriz de Confusão (RandomForestClassifier):")
fig, ax = plt.subplots(figsize=(8, 6))
ConfusionMatrixDisplay.from_predictions(
    y_test, 
    y_pred_rf, 
    ax=ax, 
    cmap='Blues',
    display_labels=['Não Atrasado', 'Atrasado']
)
plt.title('Matriz de Confusão - RandomForestClassifier (Novas Features)')
plt.show()

In [ ]:
# --- Célula 21: Importância das Features (RandomForest) ---
import pandas as pd
print("\n--- Importância das Features (Feature Importances) ---")

try:
    # 1. Pega os nomes das features DEPOIS do ColumnTransformer
    # (ex: 'num__HOUR_OF_DAY', 'cat__PART_OF_DAY_Manhã')
    feature_names = model_rf_pipe.named_steps['preprocessor'].get_feature_names_out()
    
    # 2. Pega as importâncias do classificador
    importances = model_rf_pipe.named_steps['classifier'].feature_importances_
    
    # 3. Cria um DataFrame para visualização fácil
    importance_df = pd.DataFrame({
        'Feature': feature_names,
        'Importance': importances
    }).sort_values(by='Importance', ascending=False)
    
    print("Top 15 features mais importantes para o RandomForest:")
    display(importance_df.head(15))
    
    # 4. Verificação final
    if 'num__HOUR_OF_DAY' in importance_df['Feature'].head(5).values:
         print("\n--- VEREDITO ---")
         print("SUCESSO: O RandomForest usou suas features de engenharia (HOUR_OF_DAY)!")
    else:
         print("\n--- VEREDITO ---")
         print("Interessante... o RandomForest achou outras features mais importantes.")

except Exception as e:
    print(f"Ocorreu um erro ao extrair feature importance: {e}")